## Building a company brochure generator

### We will use Gradio framework to generate a company brochure

In [1]:
# imports

import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
from langdetect import detect
#import anthropic

In [2]:
import gradio as gr # oh yeah!

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging
# We can use claude if you have the API KEY. For that, uncomment the line concerning anthropic (claude) 

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
#anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if groq_api_key:
    print(f"GROQ API Key exists and begins {groq_api_key[:8]}")
else:
    print("GROQ API Key not set")
    
#if anthropic_api_key:
 #   print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
#else:
 #   print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
GROQ API Key exists and begins gsk_fo6e
Google API Key exists and begins AIzaSyAx


In [4]:
# Connect to OpenAI, Anthropic and Google

openai = OpenAI()

#claude = anthropic.Anthropic()

google.generativeai.configure()

In [5]:
# A generic system message - no more snarky adversarial AIs!

system_message = "You are a helpful assistant"

In [6]:
# We will use GPT-4o-mini for wraping 

def message_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    completion = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
    )
    return completion.choices[0].message.content

In [7]:
# A test of this function

message_gpt("What do you think about LLMs Engineering?")

"LLM (Large Language Model) engineering is an exciting and rapidly evolving field within artificial intelligence. It involves the development, fine-tuning, and application of large language models such as GPT-3 and beyond, to solve various problems and enhance human-computer interactions. Here are some key aspects of LLM engineering:\n\n1. **Model Development**: It includes creating more sophisticated models with better architecture, training techniques, and methods to handle biases and inefficiencies present in data.\n\n2. **Fine-tuning and Customization**: LLMs can be fine-tuned on specific datasets to better serve particular applications, allowing them to perform well in niche domains without losing their general understanding.\n\n3. **Prompt Engineering**: Generating effective prompts to elicit desired responses from language models is crucial for maximizing their utility. This requires an understanding of both the model's capabilities and the nuances of language.\n\n4. **Ethics an

## User Interface using simply function

In [8]:
def _up(text):
    print(f"Shout has been called with input {text}")
    return text.upper()

In [9]:
_up("hello")

Shout has been called with input hello


'HELLO'

In [10]:
# Simply Gradio user interface
gr.Interface(fn=_up, inputs="textbox", outputs="textbox").launch()

* Running on local URL:  http://127.0.0.1:7874
* To create a public link, set `share=True` in `launch()`.


Shout has been called with input hello!


In [12]:
## Interface without Flag
gr.Interface(fn=_up, inputs="textbox", outputs="textbox", flagging_mode="never").launch()

* Running on local URL:  http://127.0.0.1:7875
* To create a public link, set `share=True` in `launch()`.


Shout has been called with input hello!


In [14]:
## To get public link, we 'll add share=True in launch()
gr.Interface(fn=_up, inputs="textbox", outputs="textbox", flagging_mode="never").launch(share=True)


* Running on local URL:  http://127.0.0.1:7877

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


Shout has been called with input hello!


In [15]:
# Adding inbrowser=True opens up a new browser window automatically

gr.Interface(fn=_up, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7878
* To create a public link, set `share=True` in `launch()`.


Shout has been called with input hello
Shout has been called with input Hello!


In [17]:
# Define this variable and then pass js=force_dark_mode when creating the Interface

force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""
gr.Interface(fn=_up, inputs="textbox", outputs="textbox", flagging_mode="never", js=force_dark_mode).launch()

* Running on local URL:  http://127.0.0.1:7879
* To create a public link, set `share=True` in `launch()`.


Shout has been called with input Hello


In [35]:
##  GPT model
def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [36]:
# Gemini model
def stream_gemini(prompt):
    gemini = google.generativeai.GenerativeModel(
        model_name='gemini-2.0-flash',
        system_instruction=system_message
    )
    result = gemini.generate_content(prompt, stream=True)
    response = ""
    for chuck in result:
        text = chuck.text
        response += text or ""
        yield response

In [37]:
##  Groq model
def stream_groq(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    groq = OpenAI(api_key = groq_api_key, base_url="https://api.groq.com/openai/v1")
    stream = groq.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [22]:
stream_groq("Hello!")

<generator object stream_groq at 0x0000027A1E9E2040>

In [9]:
# If ollama is not set up, install and do it.
import ollama

In [38]:
## Ollama model
def stream_ollama(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = ollama.chat(
        model="llama3.2",
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk['message']['content'] or ""
        yield result

In [31]:
stream_ollama("hello")

<generator object stream_ollama at 0x0000027A1E9E2840>

In [39]:
def stream_model(prompt, model):
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Gemini":
        result = stream_gemini(prompt)
    elif model == "Groq":
        result = stream_groq(prompt)
    elif model == "Ollama":
        result = stream_ollama(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [ ]:
## Try with question "What do you think about AI solution which is provide to the company?"

In [41]:
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Your message:", lines = 8), gr.Dropdown(["GPT", "Gemini", "Groq", "Ollama"], label="Select model", value="GPT")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never",
    js=force_dark_mode
)
view.launch()

* Running on local URL:  http://127.0.0.1:7888
* To create a public link, set `share=True` in `launch()`.


## Brochure generator

In [58]:
# A class to represent a Webpage

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [59]:
## Model role
system_message1 = "You are an assistant that analyzes the contents of a company website landing page \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Use the same language like what is use on the landing page."

system_message2 = system_message1 + "Use a sympathetic and empathetic tone"

system_message = system_message2

In [60]:
def stream_brochure(company_name, url, model):
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += Website(url).get_contents()
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Gemini":
        result = stream_gemini(prompt)
    elif model == "Groq":
        result = stream_groq(prompt)
    elif model == "Ollama":
        result = stream_ollama(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [61]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company name:", lines = 2),
        gr.Textbox(label="Landing page URL including http:// or https://", lines = 2),
        gr.Dropdown(["GPT", "Gemini", "Groq", "Ollama"], label="Select model")],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never",
    js=force_dark_mode
)
view.launch()

* Running on local URL:  http://127.0.0.1:7893
* To create a public link, set `share=True` in `launch()`.
